In [1]:
import numpy as np 
import torch
import torch.nn as nn
import pandas as pd
pd.options.display.max_columns = None
from tqdm import tnrange
from sklearn.metrics import accuracy_score

In [2]:
raw = pd.read_csv('iris.data', header=None, names=['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])

In [3]:
raw.shape

(150, 5)

In [4]:
raw.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
raw['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [6]:
class model(nn.Module):
    def __init__(self, outputdim=3):
        super(model, self).__init__()

        # some random parameter
        self.outputdim = outputdim
        
        # neural network
        self.dense1 = nn.Linear(4, 10)
        self.relu1 = nn.ReLU()
        self.dense2 = nn.Linear(10, self.outputdim)
        self.soft = nn.Softmax()
        
    def forward(self, x):
        x = self.dense1(x)
        x = self.relu1(x)
        x = self.dense2(x)
        x = self.soft(x)
        
        return x

In [7]:
net = model(outputdim=3)

# test one random sample through the network

In [8]:
data = torch.Tensor(np.random.randint(1, 10, (4)))
data

tensor([7., 2., 5., 5.])

In [9]:
net(data)

/Users/gram/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([0.1936, 0.6801, 0.1263], grad_fn=<SoftmaxBackward>)

# tensorfy data

In [10]:
# shuffle input data
raw = raw.sample(frac=1).reset_index(drop=True)

In [11]:
X = raw[['sepal length', 'sepal width', 'petal length', 'petal width']]
X = X.values
X = torch.Tensor(X)

X.shape

torch.Size([150, 4])

In [12]:
y = raw['class']
y = pd.get_dummies(y)
y = y.values
y = torch.LongTensor(y)

y.shape

torch.Size([150, 3])

In [13]:
y = raw['class']
y = y.replace(to_replace=y.unique(), value=[0,1,2])
y = y.values
y = torch.LongTensor(y)

y.shape

torch.Size([150])

# set up the tools needed for training

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

# training run

In [17]:
for epoch in range(10000):
    preds = net(X)
    
    loss = criterion(preds, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(epoch)

/Users/gram/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [18]:
valpreds = net(X).data.numpy()
valpreds = np.argmax(valpreds, axis=1)
vallabels = y.data.numpy()

/Users/gram/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [19]:
accuracy_score(vallabels, valpreds)

0.8533333333333334